# SISTEMA DE RECUPERACIÓN DE INFORMACIÓN
 - CORPUS: Noticias BBC

In [41]:
pip install nltk pandas scikit-learn tabulate rank_bm25

In [42]:
# Importación de Módulo
import csv
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize
from tabulate import tabulate
from collections import defaultdict, Counter
import math
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from rank_bm25 import BM25Okapi

In [43]:
# Parte 0: Carga del corpus
""" Leer un corpus de documentos en texto plano."""

def cargar_corpus(ruta):
    """Cargar corpus desde archivo CSV"""
    print("Cargando corpus de noticias BBC...")

    corpus = []

    with open(ruta, 'r', encoding='utf-8') as archivo:
        contenido = csv.reader(archivo)
        next(contenido)  # No lee el encabezado salta a la siguiente linea

        for f in contenido:
            corpus.append(f[-1]) #Mostramos la ultima columna "description"

    print(f"Corpus cargado: {len(corpus)} documentos\n")
    return corpus

def mostrar_info_dataset(corpus,n=10):
    """Mostrar información del dataset"""
    if not corpus:
        print("Error: No hay corpus cargado")
        return

    print("\n" + "="*80)
    print("INFORMACIÓN DEL DATASET".center(80))
    print()
    print(f"Total de documentos: {len(corpus)}")
    print(f"\nContenido del documento:")

    for f, doc in enumerate(corpus[:n], start=1):
        print(f"{f:04d}. {doc}")
    print("...")
    for f, doc in enumerate(corpus[-n:], start=len(corpus)-n+1):
        print(f"{f:04d}. {doc}")

    print("\n")



In [44]:
# Descargar recursos NLTK necesarios si no están disponibles
recursos_nltk = ['punkt', 'stopwords','punkt_tab']
for recurso in recursos_nltk:
    try:
        # Intenta encontrar los recursos
        if 'punkt' in recurso:
            nltk.data.find(f'tokenizers/{recurso}')
        else:
            nltk.data.find(f'corpora/{recurso}')
    except LookupError:
        # Si no están disponibles, los descarga
        print(f"Descargando {recurso}...")
        nltk.download(recurso, quiet=True)


In [45]:
# Parte 1: Procesamiento básico: tokenización, normalización y remoción de stopwords

#------------------------------------------------------------------------------------------------------
#Preprocesamiento
def inicializar_preprocesamiento():
    """Inicializar herramientas de preprocesamiento"""
    return {
        'stemmer': PorterStemmer(),
        'lemmatizer': WordNetLemmatizer(),
        'stop_words': set(stopwords.words('english'))
    }

def tokenizar(documento):
    """Tokenización del texto"""
    doc_tokenizacion = word_tokenize(documento.lower())
    return doc_tokenizacion

def normalizar(tokenizacion):
    """Normalización: remover puntuación y caracteres especiales"""
    doc_normalizado = [token for token in tokenizacion if token.isalpha()]
    return doc_normalizado

def remover_stopwords(tokenizacion, stop_words):
    """Remoción de stopwords"""
    doc_sin_stopwords = [token for token in tokenizacion if token not in stop_words]
    return doc_sin_stopwords

def stemming(tokenizacion, stemmer):
    """Aplicacion de stemming (reducción a raíz)"""
    doc_stemmed = [stemmer.stem(token) for token in tokenizacion]
    return doc_stemmed

def procesamiento_doc(documento, preprocesamientod):
    """Pipeline completo de procesamiento"""
    # 1. Tokenización
    doc_tokenizacion = tokenizar(documento)

    # 2. Normalización
    doc_normalizacion = normalizar(doc_tokenizacion)

    # 3. Remover stopwords
    doc_sin_stopwords = remover_stopwords(doc_normalizacion, preprocesamientod['stop_words'])

    # 4. Stemming
    doc_stemmed = stemming(doc_sin_stopwords, preprocesamientod['stemmer'])

    return doc_stemmed

In [46]:
# Construcción índice invertido
def construir_indice(doc_preprocesados):
    """Construcción del indice invertido"""
    indice_invertido = {}

    for doc_id, doc in enumerate(doc_preprocesados):
        # Si doc es una lista de tokens, unirlos
        if isinstance(doc, list):
            doc = ' '.join(doc)

        if not doc.strip():
            continue

        term_freq = Counter(doc.split())

        for term, freq in term_freq.items():
            if term not in indice_invertido:
                indice_invertido[term] = {}
            indice_invertido[term][doc_id] = freq

    #print(f" Índice invertido construido con {len(indice_invertido)} términos únicos")
    return indice_invertido

def mostrar__indice_invertido(indice_invertido, num_terms=15):
    print("\n" + "="*80)
    print(f"ÍNDICE INVERTIDO".center(80))
    print("="*80)

    term_data = []


    for term, doc_freqs in indice_invertido.items():
        total_freq = sum(doc_freqs.values())
        num_docs = len(doc_freqs)

        # Lista de documentos
        docs = list(doc_freqs.keys())
        # Mostrar solo primeros 10 documentos
        if len(docs) > 5:
            docs_str = str(docs[:5])[:-1] + ", ...]"
        else:
            docs_str = str(docs)

        term_data.append({
            'Término': term,
            'Frecuencia_Total': total_freq,
            'Num_Documentos': num_docs,
            'Frecuencia_Promedio': round(total_freq / num_docs, 2),
            'Documentos': docs_str
        })


    df_terminos = pd.DataFrame(term_data)
    df_terminos = df_terminos.sort_values('Frecuencia_Total', ascending=False)


    print(f"Total de términos únicos en el índice: {len(indice_invertido)}")
    print()

    print("\n" + "="*80)
    print(f"RESUMEN DE LOS {min(num_terms, len(df_terminos))} TÉRMINOS MÁS FRECUENTES:")
    print("="*80)
    print(tabulate(df_terminos.head(num_terms), headers='keys', tablefmt='fancy_grid', showindex=False))

    print("\n" + "="*80)
    print(f"RESUMEN DE LOS {min(num_terms, len(df_terminos))} TÉRMINOS MENOS FRECUENTES:")
    print("="*80)
    print(tabulate(df_terminos.tail(num_terms), headers='keys', tablefmt='fancy_grid', showindex=False))


def mostrar_tabla_documentos(corpus, doc_procesados, num_docs=20):
    num_docs = min(num_docs, len(corpus))

    table_data = []
    for idx in range(num_docs):
        original_text = corpus[idx][:100].replace('\n', ' ').replace('\t', ' ')
        if len(corpus[idx]) > 100:
            original_text += "..."

        # Convertir lista de tokens a string si es necesario
        if isinstance(doc_procesados[idx], list):
            processed_text = ' '.join(doc_procesados[idx])[:100]
        else:
            processed_text = doc_procesados[idx][:100]

        processed_text = processed_text.replace('\n', ' ').replace('\t', ' ')
        if len(str(doc_procesados[idx])) > 100:
            processed_text += "..."

        table_data.append([
            idx + 1,
            original_text.strip(),
            processed_text.strip()
        ])

    headers = [
        "N° Doc",
        "Texto Original ",
        "Texto Preprocesado "
    ]

    print("\n TABLA COMPARATIVA: DOCUMENTOS ORIGINALES VS PREPROCESADOS")
    print("=" * 120)
    print(f"\nMostrando {num_docs} de {len(corpus)} documentos disponibles")
    print(tabulate(table_data, headers=headers, tablefmt="grid", maxcolwidths=[4, 45, 50]))

In [47]:
# Parte 2: Modelos de recuperación de información
# Modelo TF-IDF
def construir_modelo_tfidf(docs_procesados):
    print("Construyendo modelo TF-IDF...")

    # Convertir listas de tokens a strings
    docs_cadenas = []
    for doc in docs_procesados:
        if isinstance(doc, list):
            docs_cadenas.append(' '.join(doc))
        else:
            docs_cadenas.append(doc)

    vectorizador_tfidf = TfidfVectorizer()
    matriz_tfidf = vectorizador_tfidf.fit_transform(docs_cadenas) # Aquí se calcula TF e IDF

    print("Modelo TF-IDF construido")
    return vectorizador_tfidf, matriz_tfidf

def buscar_tfidf(vectorizador_tfidf, matriz_tfidf, consulta_procesada_str):
    """
    Realiza búsqueda usando TF-IDF y similitud coseno.

    consulta_procesada_str debe ser un string (la consulta limpia).
    """
    # 1. Vectorizar la consulta (Vector Query TF-IDF)
    query_vector = vectorizador_tfidf.transform([consulta_procesada_str])

    # 2. Calcular la Similitud Coseno
    similitud_coseno = cosine_similarity(query_vector, matriz_tfidf).flatten()

    resultados_similitud = [(i, score) for i, score in enumerate(similitud_coseno) if score > 0]
    resultados_similitud.sort(key=lambda x: x[1], reverse=True)

    return resultados_similitud

In [48]:
# Modelo BM25
def construir_modelo_bm25(docs_procesados):
    print("Construyendo modelo BM25...")

    # Asegurar que tenemos listas de tokens
    tokenized_docs = []
    for doc in docs_procesados:
        if isinstance(doc, list):
            tokenized_docs.append(doc)
        else:
            tokenized_docs.append(doc.split())

    modelo_bm25 = BM25Okapi(tokenized_docs)
    print("Modelo BM25 construido")
    return modelo_bm25

def buscar_bm25(modelo_bm25, consulta_procesada_str):
    """
    Realiza búsqueda usando BM25.

    consulta_procesada_str es un string, pero debe tokenizarse.
    """
    # Tokenizar la consulta procesada
    query_tokens = consulta_procesada_str.split()

    # Calcular los scores
    doc_scores = modelo_bm25.get_scores(query_tokens)

    resultados_similitud = [(i, score) for i, score in enumerate(doc_scores) if score > 0]
    resultados_similitud.sort(key=lambda x: x[1], reverse=True)

    return resultados_similitud


In [49]:
# Modelo Jaccard
def buscar_jaccard(docs_procesados, consulta_procesada_str):
    """
    Realiza búsqueda usando la similitud Jaccard.

    docs_procesados: Lista de listas de tokens (corpus preprocesado)
    consulta_procesada_str: String de la consulta preprocesada
    """
    consulta_set = set(consulta_procesada_str.split())
    resultados_similitud = []

    for doc_id, doc_tokens in enumerate(docs_procesados):
        doc_set = set(doc_tokens)

        # Calcular Similitud Jaccard: |A ∩ B| / |A ∪ B|
        interseccion = len(consulta_set.intersection(doc_set))
        union = len(consulta_set.union(doc_set))

        score = interseccion / union if union > 0 else 0

        if score > 0:
            resultados_similitud.append((doc_id, score))

    resultados_similitud.sort(key=lambda x: x[1], reverse=True)

    return resultados_similitud



In [50]:
def ejecutar_consultas(vectorizador_tfidf, matriz_tfidf,modelo_bm25,corpus, docs_procesados, procesamiento, nombre_metodo):
    while True:
        print(f"\nBÚSQUEDA CON {nombre_metodo}")
        print("=" * 60)
        query_original = input("Ingrese la  consulta (o digite 'salir' para volver al menú): ").strip()

        '''if query_original.lower() == 'salir':
            break
        if not query_original:
            print(" Por favor ingresa una consulta válida.")
            return'''

        # 1. Procesar la consulta (limpieza, stopwords, stemming, etc.)
        procesar_consulta = procesamiento_doc(query_original,procesamiento)

        # 2. Convertir la consulta procesada (lista de tokens) a string
        consulta_procesada_str = ' '.join(procesar_consulta)

        if not consulta_procesada_str.strip():
            print(" La consulta no contiene términos válidos después del procesamiento.")
            continue

        resultados = []
        if nombre_metodo == "TF-IDF":
            resultados = buscar_tfidf(vectorizador_tfidf, matriz_tfidf, consulta_procesada_str)
        elif nombre_metodo == "BM25":
            resultados = buscar_bm25(modelo_bm25, consulta_procesada_str)
        elif nombre_metodo == "Jaccard":
            resultados = buscar_jaccard(docs_procesados, consulta_procesada_str)
        if not resultados:
            print(" No se encontraron documentos relevantes.")
            continue



        # Resultados
        print(f"\nRESULTS {nombre_metodo} PARA: '{query_original}'")
        print(f"Se encontraron {len(resultados)} documentos relevantes")
        print("=" * 60)


        for rank, (doc_id, score) in enumerate(resultados[:10], start=1):
            print(f"\nRESULTADO #{rank}")
            print(f"   Score {nombre_metodo}: {score:.4f}")
            print(f"   ID Documento: {doc_id}")


            content = corpus[doc_id][:200].replace('\n', ' ')
            print(f"   Contenido: {content}...")

            if rank < 5 and rank < len(resultados):
                print("   " + "─" * 50)


        if len(resultados) > 5:
            print(f"\n ... y {len(resultados) - 5} documentos más")
        break


In [51]:
# Parte 3: Evaluación del sistema

def cargar_consultas_evaluacion():
    """Retorna los textos de consulta y sus documentos relevantes (qrels)"""
    queries = {
        'Q1': "Ukraine's youngest cabinet minister",
        'Q2': 'Russian gymnast Ivan Kuliak is being investigated',
        'Q3': 'The Ukrainian president says the country will ',
        'Q4': 'TikTok suspends live streaming',
        'Q5': 'The FIA wants to limit the ways its'
    }

    qrels = {
        'Q1': [13, 36037, 13511, 7629, 14249],
        'Q2': [11, 78, 53, 104, 3030],
        'Q3': [0, 5527, 2648, 12267, 1372],
        'Q4': [7, 31437, 2474, 15057, 23740],
        'Q5': [42109, 19020, 31481, 31311, 28727]
    }
    return queries, qrels

def calcular_precision_recall(resultados_ranking, documentos_relevantes):
    """
    Calcula Precisión y Exhaustividad (Recall) para una consulta dada.

    resultados_ranking: Lista de IDs de documentos recuperados por el modelo, ordenados por score.
    documentos_relevantes: Set de IDs de documentos que son relevantes (QRELS).
    """
    relevantes_encontrados = 0
    precisiones_en_relevantes = []

    # Total de documentos relevantes
    R = len(documentos_relevantes)
    if R == 0:
        return 0, 0, 0 # P, R, AP (Average Precision)

    # Cálculo de Precisión, Recall y AP
    for k, doc_id in enumerate(resultados_ranking, start=1):
        if doc_id in documentos_relevantes:
            relevantes_encontrados += 1
            # 1. Precisión en el corte k: (relevantes encontrados hasta k) / k
            precision_en_k = relevantes_encontrados / k
            precisiones_en_relevantes.append(precision_en_k)

        # Opcional: Detener si encontramos todos los relevantes, optimizando el cálculo
        if relevantes_encontrados == R:
             break

    # 2. Precisión y Exhaustividad final (calculada en el corte máximo)
    if not resultados_ranking:
        P = 0
    else:
        P = relevantes_encontrados / len(resultados_ranking)

    R_score = relevantes_encontrados / R # Recall: R_encontrados / R_total

    # 3. Average Precision (AP): Suma de P@k / R
    AP = sum(precisiones_en_relevantes) / R if R > 0 else 0

    return P, R_score, AP

def evaluar_sistema(modelos_busqueda, procesamiento):
    """
    Realiza la evaluación de los modelos usando las QRELS predefinidas,
    mostrando una tabla de resultados por cada modelo (TF-IDF, BM25, Jaccard).
    """
    # 1. Cargar QRELS correctamente
    queries_dict, qrels_dict = cargar_consultas_evaluacion()

    # Lista para almacenar todos los resultados (luego se filtrará por modelo)
    resultados_eval = []

    # 3. Iterar sobre los modelos (TF-IDF, BM25, Jaccard)
    for nombre_metodo in ["TF-IDF", "BM25", "Jaccard"]:
        print(f"\n{'='*70}")
        print(f"--- EVALUANDO MODELO: {nombre_metodo} ---")
        print(f"{'='*70}")

        suma_ap = 0

        # 4. Iterar sobre el diccionario de TEXTO de consultas
        for q_id, query_texto in queries_dict.items():

            # Obtener los relevantes para este ID de consulta
            # Nota: Asumimos que todos los IDs de queries_dict están en qrels_dict
            docs_relevantes = qrels_dict.get(q_id, [])

            # 4a. Procesar la consulta
            procesar_consulta = procesamiento_doc(query_texto, procesamiento)
            consulta_procesada_str = ' '.join(procesar_consulta)

            if not consulta_procesada_str.strip():
                continue

            # 4b. Ejecutar la búsqueda
            resultados = []
            if nombre_metodo == "TF-IDF":
                # La búsqueda devuelve (ID, score)
                resultados = buscar_tfidf(modelos_busqueda['vectorizador_tfidf'], modelos_busqueda['matriz_tfidf'], consulta_procesada_str)
            elif nombre_metodo == "BM25":
                resultados = buscar_bm25(modelos_busqueda['modelo_bm25'], consulta_procesada_str)
            elif nombre_metodo == "Jaccard":
                # Importante: Usar el corpus procesado para Jaccard
                resultados = buscar_jaccard(modelos_busqueda['doc_procesados'], consulta_procesada_str)

            # Extraer IDs
            ranking_modelo = [doc_id for doc_id, score in resultados]

            # 4c. Calcular métricas
            P, R_score, AP = calcular_precision_recall(ranking_modelo, set(docs_relevantes))

            # 4d. Almacenar resultados
            resultados_eval.append({
                'Modelo': nombre_metodo,
                'Consulta_ID': q_id, # Usamos ID para la tabla
                'Precisión': f"{P:.4f}",
                'Exhaustividad': f"{R_score:.4f}",
                'AP': AP
            })
            suma_ap += AP

        # 5. Calcular MAP para el modelo
        # Usamos len(queries_dict) para el denominador (total de consultas)
        MAP = suma_ap / len(queries_dict)

        # 6. Preparar y mostrar la tabla para el modelo actual

        # Filtrar solo los resultados del modelo actual
        df_modelo = pd.DataFrame([r for r in resultados_eval if r['Modelo'] == nombre_metodo and r['Consulta_ID'] != '--- MAP ---'])

        # Añadir la fila de MAP para la impresión
        map_row = pd.DataFrame([{
            'Modelo': nombre_metodo,
            'Consulta_ID': '--- MAP ---',
            'Precisión': '',
            'Exhaustividad': '',
            'AP': f"{MAP:.4f}"
        }])

        df_final_modelo = pd.concat([df_modelo, map_row], ignore_index=True)

        # Mostrar la tabla separada
        print(tabulate(df_final_modelo[['Consulta_ID', 'Precisión', 'Exhaustividad', 'AP']],
                       headers=['Consulta ID', 'Precisión', 'Exhaustividad (Recall)', 'AP'],
                       tablefmt='fancy_grid',
                       showindex=False))


def mostrar_queries_qrels():
    """Muestra las consultas y los documentos relevantes asociados (QRELS) en formato de tabla."""
    queries, qrels = cargar_consultas_evaluacion()

    print("\n" + "="*80)
    print("CONSULTAS DE EVALUACIÓN (QUERIES) Y DOCUMENTOS RELEVANTES (QRELS)".center(80))
    print()

    table_data = []

    for q_id in queries:
        query_text = queries[q_id]
        # Formatear la lista de QRELs: [1, 2, 3, 4, 5] -> "1, 2, 3, 4, 5"
        qrels_list = qrels.get(q_id, [])
        qrels_str = ", ".join(map(str, qrels_list))

        table_data.append([
            q_id,
            query_text,
            qrels_str
        ])

    headers = ["ID Consulta", "Texto de la Consulta", "IDs Documentos Relevantes (QRELs)"]

    print(tabulate(table_data, headers=headers, tablefmt="fancy_grid", maxcolwidths=[10, 35, 30]))


In [52]:
# SISTEMA DE RECUPERACIÓN DE INFORMACIÓN
#----------------------------------------------
def Sistema_RI():
    "Menú principal del sistema"

    # Cargar del corpus
    ruta = r"/content/bbc_news.csv"
    corpus = cargar_corpus(ruta)

    # Preprocesameinto
    print("\nInicializando preprocesameinto de documentos...")
    preprocesamiento = inicializar_preprocesamiento()
    doc_procesados=[procesamiento_doc(doc,preprocesamiento)for doc in corpus]
    print("Preprocesamiento completado")

    #Contruccion del indice
    print("\nConstruyendo el índice invertido...")
    indice_invertido = construir_indice(doc_procesados)
    print("Índice construido\n")

    vectorizador_tfidf, matriz_tfidf = construir_modelo_tfidf(doc_procesados)
    modelo_bm25 =construir_modelo_bm25(doc_procesados)



    print("\n Sistema listo!")

    while True:
        print("\n" + "="*80)
        print("SISTEMA DE RECUPERACIÓN DE INFORMACIÓN".center(80))
        print("="*80)
        print("1. Ver información del dataset")
        print("2. Ver índice invertido")
        print("3. Mostrar tabla de documentos (original vs procesado)")
        print("4. Búsqueda TF-IDF")
        print("5. Búsqueda BM25")
        print("6. Búsqueda Jaccard")
        print("7. Evaluación de resultados")
        print("8. Ver queries y QRELs")
        print("9. Salir")
        print()


        op = int(input("Eliga una opción: "))
        if op == 1:
            print("Mostrando información del dataset...")
            mostrar_info_dataset(corpus)
        elif op == 2:
            mostrar__indice_invertido(indice_invertido)
        elif op == 3:
            mostrar_tabla_documentos(corpus,doc_procesados)
        elif op== 4:
            ejecutar_consultas(vectorizador_tfidf, matriz_tfidf ,modelo_bm25,corpus,doc_procesados, preprocesamiento, "TF-IDF")
        elif op == 5:
            ejecutar_consultas(vectorizador_tfidf, matriz_tfidf,modelo_bm25, corpus,doc_procesados, preprocesamiento, "BM25")
        elif op == 6:
            ejecutar_consultas(vectorizador_tfidf, matriz_tfidf,modelo_bm25, corpus,doc_procesados, preprocesamiento, "Jaccard")
            print()
        elif op == 7:
            # Crear un diccionario de recursos para pasar a la función de evaluación
            modelos_busqueda = {
                'vectorizador_tfidf': vectorizador_tfidf,
                'matriz_tfidf': matriz_tfidf,
                'modelo_bm25': modelo_bm25,
                'doc_procesados': doc_procesados # ¡Necesario para Jaccard en evaluación!
            }
            evaluar_sistema(modelos_busqueda, preprocesamiento)
        elif op == 8:
            mostrar_queries_qrels()
        elif op == 9:
            print("Saliendo del sistema. ¡Hasta luego!")
            break
        else:
            print("Opción no válida. Intente nuevamente.")

In [53]:
Sistema_RI()

Cargando corpus de noticias BBC...
Corpus cargado: 42115 documentos


Inicializando preprocesameinto de documentos...
Preprocesamiento completado

Construyendo el índice invertido...
Índice construido

Construyendo modelo TF-IDF...
Modelo TF-IDF construido
Construyendo modelo BM25...
Modelo BM25 construido

 Sistema listo!

                     SISTEMA DE RECUPERACIÓN DE INFORMACIÓN                     
1. Ver información del dataset
2. Ver índice invertido
3. Mostrar tabla de documentos (original vs procesado)
4. Búsqueda TF-IDF
5. Búsqueda BM25
6. Búsqueda Jaccard
7. Evaluación de resultados
8. Ver queries y QRELs
9. Salir

Eliga una opción: 1
Mostrando información del dataset...

                            INFORMACIÓN DEL DATASET                             

Total de documentos: 42115

Contenido del documento:
0001. The Ukrainian president says the country will not forgive or forget those who murder its civilians.
0002. Jeremy Bowen was on the frontline in Irpin, as residents came